# Word Embedding

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
nltk.download('punkt')

from gensim.models import Word2Vec, FastText, KeyedVectors

import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import argparse
import wandb
wandb.init(project="DACON_235938", name="word_embedding")

parser = argparse.ArgumentParser(description="word_embedding")
parser.add_argument('--embedding_model', default="fasttext", type=str) # word2vec or fasttext
parser.add_argument('--embedding_size', default=100, type=int)
parser.add_argument('--embedding_workers', default=30, type=int)
parser.add_argument('--feature_name', default="tokenized_stem", type=str) # reviews or tokenized_stem or main_pos
parser.add_argument('--validation_size', default=0.2, type=float)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

embedding_model = args.embedding_model
embedding_size = args.embedding_size
embedding_workers = args.embedding_workers
feature_name = args.feature_name
validation_size = args.validation_size
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seeds()

train = pd.read_csv("data/train_okt.csv")
test = pd.read_csv("data/test_okt.csv")

train.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Taehyeong\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,id,reviews,target,tokenized_stem,main_pos
0,0,조아요 처음구입 싸게햇어요,2,조아요 처음 구입 싸게 햇 어 요,조아요 처음 구입 싸게 햇 요
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1,생각 보다 잘 안 돼요 매지 바른지 하루 밖에 안 됐는데 ㅠㅠ 25천원 가량 주고 ...,생각 보다 잘 안 돼요 매지 바른지 하루 안 됐는데 ㅠㅠ 25천원 가량 주고 사기 ...
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...,디자인 은 괜찮은데 상품 이 금 이 가서 교환 했는데 두번째 받은 상품 도 까져있고...
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2,기전 에 이 제품 말고 이마트 트 레이더스 에서만 팔던 프리미엄 제품 을 사용 했었...,기전 이 제품 이마트 트 레이더스 팔던 프리미엄 제품 사용 했었습니다 샘플 써 보고...
4,4,튼튼하고 손목을 잘 받쳐주네요~,5,튼튼하고 손목 을 잘 받쳐주네요 ~,튼튼하고 손목 잘 받쳐주네요


## Preprocessing

In [2]:
y = train["target"]

y.value_counts()

5    10000
2     8000
1     4500
4     2500
Name: target, dtype: int64

In [3]:
def preprocessing_text(df):
    temp = df.copy()
    df_token = []
    for i in range(len(temp)):
        df_token.append(nltk.word_tokenize(temp[i]))
    return df_token

X = np.array(preprocessing_text(train[feature_name]))
X_test = np.array(preprocessing_text(test[feature_name]))

X.shape, X_test.shape

((25000,), (25000,))

In [4]:
if embedding_model == "word2vec":
    model = Word2Vec(sentences=X, size=embedding_size,
                     workers=embedding_workers, seed=seed, sg=1)
if embedding_model == "fasttext":
    model = FastText(sentences=X, size=embedding_size,
                     workers=embedding_workers, seed=seed, sg=1)

X.shape, X_test.shape

((25000,), (25000,))

In [5]:
model.wv.most_similar('좋아요')

[('딱좋아요', 0.8901451826095581),
 ('좋네요', 0.8676313161849976),
 ('좋습니다', 0.8392004370689392),
 ('조아요', 0.8371558785438538),
 ('편해요', 0.8126579523086548),
 ('좋음', 0.7972337007522583),
 ('좋구요', 0.793621838092804),
 ('좋아용', 0.7924681901931763),
 ('좋아여', 0.7922549247741699),
 ('만족합니당', 0.7911874055862427)]

In [6]:
model.wv.most_similar('안좋아요')

[('안좋고', 0.9053457975387573),
 ('안좋음', 0.9034500122070312),
 ('많아요', 0.8645927906036377),
 ('낮아요', 0.8574504852294922),
 ('받아요', 0.8555399775505066),
 ('높아요', 0.8541637659072876),
 ('밝아요', 0.8512791395187378),
 ('예쁜데', 0.8499303460121155),
 ('부실해요', 0.8451288342475891),
 ('안좋네요', 0.8432906270027161)]

In [7]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)
    num_words = 0
    index2word_set = set(model.wv.index2word)
    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[w])
    return feature_vector

def get_dataset(reviews, model, num_features):
    dataset = list()
    for s in reviews:
        dataset.append(get_features(s, model, num_features))
    reviewFeatureVecs = np.stack(dataset)
    return reviewFeatureVecs

X = get_dataset(X, model, embedding_size)
X_test = get_dataset(X_test, model, embedding_size)

X.shape, X_test.shape

((25000, 100), (25000, 100))

## Training

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=validation_size, random_state=seed, stratify=y
)

model = LogisticRegression(solver="liblinear", random_state=seed, n_jobs=-1) 
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
   
val_acc = accuracy_score(y_val, y_pred)
print(f"val_acc : {val_acc}")
wandb.log({"val_acc": val_acc})

val_acc : 0.6468


In [9]:
model = LogisticRegression(solver="liblinear", random_state=seed, n_jobs=-1) 
model.fit(X, y)
prediction = model.predict(X_test)

pd.Series(prediction).value_counts()

5    12429
2    10921
1     1586
4       64
dtype: int64

## Inference

In [10]:
submission = pd.read_csv("data/sample_submission.csv")
submission["target"] = prediction
submission.to_csv("submission.csv", index=False)